# Exercise 8.9 {.unnumbered}
> This problem involves the `OJ` data set which is part of the ISLP
package.


In [ ]:
#| echo: False
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
oj = pd.read_csv("data/OJ.csv")
oj.head()

## (a)
> Create a training set containing a random sample of 800 observations, and a test set containing the remaining observations


In [ ]:
from sklearn.model_selection import train_test_split

X = oj.drop("Purchase", axis=1).replace({"Store7":{"Yes": 1, "No": 0}})
y = oj["Purchase"].replace({"CH": 1, "MM": 0})

X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=0, train_size=0.8
)

## (b)
> Fit a tree to the training data, with Purchase as the response and the other variables as predictors. What is the training error rate?


In [ ]:
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier(random_state=0, max_leaf_nodes=5)
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

## (c)

In [ ]:
from sklearn.tree import plot_tree
plot_tree(clf)

The tree has $5$ terminal nodes (`clf.get_n_leaves()`). 


In [ ]:
clf.get_n_leaves()

## (e)
> Predict the response on the test data, and produce a confusion matrix comparing the test labels to the predicted test labels. What is the test error rate?


In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay
ConfusionMatrixDisplay.from_estimator(clf, X_test, y_test)

In [ ]:
from sklearn.metrics import RocCurveDisplay
RocCurveDisplay.from_estimator(clf, X_test, y_test)

## (f)
> Use cross-validation on the training set in order to determine the optimal tree size.


In [ ]:
from sklearn.model_selection import GridSearchCV

clf_cv = GridSearchCV(
    DecisionTreeClassifier(random_state=0),
    param_grid=[{"max_leaf_nodes": range(2, 20)}],
    cv=5,
    refit="neg_log_loss",
    scoring=["neg_log_loss", "accuracy"],
)

clf_cv.fit(X_train, y_train)
clf_cv.score(X_test, y_test)

***Note:*** The score method is inconsistent here. To get accuracy we need


In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(clf_cv.predict(X_test), y_test)

## (g)
> Produce a plot with tree size on the $x$-axis and cross-validated
log loss on the $y$-axis.


In [ ]:
x = [list(x.values())[0] for x in clf_cv.cv_results_["params"]]
y = clf_cv.cv_results_["mean_test_score"]
plt.plot(x,y)
plt.xlabel("Maximal number of leaf nodes")
plt.ylabel("Cross-validated negative log loss")

## (with pipelines)

In [ ]:
X = oj.drop("Purchase", axis=1)
y = oj["Purchase"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=0, train_size=0.8
)
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import TargetEncoder
from sklearn.compose import ColumnTransformer

clf = make_pipeline(
    ColumnTransformer(
        [("Store7", OneHotEncoder(), ["Store7"])],
        remainder="passthrough",
    ),
    DecisionTreeClassifier(random_state=0),
)
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

## (c)
> Create a plot of the tree, and interpret the results. How many terminal nodes does the tree have?


In [ ]:
plot_tree(clf.named_steps["decisiontreeclassifier"])

In [ ]:
clf.get_n_leaves()

## (d)
> Predict the response on the test data, and produce a confusion matrix comparing the test labels to the predicted test labels. What is the test error rate?
